In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, push_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.themes import Theme

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
plot_theme = Theme("./theme.yml")

In [4]:
def f(x, c):
    return np.sin(4 * x) + c

In [5]:
data_size = 1000

In [6]:
x = np.linspace(-np.pi/8, np.pi/4, data_size)
y = f(x, 2) + np.random.normal(0, 0.5, data_size)

In [7]:
p = figure(plot_width=600, plot_height=600)
p.circle(x, y, size=10, alpha=0.2, color="#66D9EF", legend="y")
p.line(x, f(x, 2), color="#F92672", line_width=3, legend="Actual", line_dash="dashed")
doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

In [8]:
def gaussian_kernel(x_, x0_, h_):
    return np.exp(- 0.5 * np.power((x_ - x0_) / h_, 2) )

# One Dimentional Kernel Smoother

In [9]:
def predict(x_test, x_train, y_train, bandwidth, kernel_func=gaussian_kernel):
    return np.array([(kernel_func(x_train, x0, bandwidth).dot(y_train) ) / 
                     kernel_func(x_train, x0, bandwidth).sum() for x0 in x_test])

In [10]:
h_values = [0.01, 0.1, 1]
colors = ["#A6E22E", "#FD971F", "#AE81FF"]

In [11]:
p = figure(plot_width=600, plot_height=600)
p.circle(x, y, size=10, alpha=0.2, color="#66D9EF", legend="y")
p.line(x, f(x, 2), color="#F92672", line_width=3, legend="Actual", line_dash="dashed")

for idx, h in enumerate(h_values):
    p.line(x, predict(x, x, y, h), color=colors[idx], line_width=1, legend="y_hat (h={})".format(h))
    
p.title.text = "Kernel Regression (Gaussian)"
p.xaxis.axis_label = "x"
p.yaxis.axis_label = "f(x)"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

In [12]:
h_range = np.linspace(0.01, 0.2, 20)
mses = [np.mean(np.power(y - predict(x, x, y, h), 2)) for h in h_range]

In [13]:
p = figure(plot_width=600, plot_height=300)
p.circle(x=h_range, y=mses, size=10, color="#66D9EF")
p.line(x=h_range, y=mses, color="#66D9EF", line_width=3)

p.title.text = "MSE vs Backwidth"
p.xaxis.axis_label = "Backwidth"
p.yaxis.axis_label = "MSE"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

# Cross Validation

## Leave One Out Cross Validation (LOOCV)

In [41]:
mse_values = []

for h in h_range:
    print("h = {}".format(h))
    errors = []
    for idx, val in enumerate(x):
        x_test = np.array([val])
        y_test = np.array([y[idx]])
        x_train = np.append(x[:idx], x[idx+1:])
        y_train = np.append(y[:idx], y[idx+1:])
        assert len(x_train) == data_size - 1
        y_test_hat = predict(x_test, x_train, y_train, h)
        errors.append((y_test_hat - y_test)[0])
    mse_values.append(np.mean(np.power(errors, 2)))

h = 0.01
h = 0.02
h = 0.03
h = 0.04
h = 0.05
h = 0.06
h = 0.07
h = 0.08
h = 0.09
h = 0.1
h = 0.11
h = 0.12
h = 0.13
h = 0.14
h = 0.15
h = 0.16
h = 0.17
h = 0.18
h = 0.19
h = 0.2


In [42]:
p = figure(plot_width=600, plot_height=300)
p.circle(x=h_range, y=mse_values, size=10, color="#66D9EF")
p.line(x=h_range, y=mse_values, color="#66D9EF", line_width=3)

p.title.text = "MSE vs Backwidth"
p.xaxis.axis_label = "Backwidth"
p.yaxis.axis_label = "MSE"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

In [43]:
h_optimal = h_range[np.argmin(mse_values)]
print(h_optimal)

0.04


In [44]:
p = figure(plot_width=600, plot_height=600)
p.circle(x, y, size=10, alpha=0.2, color="#66D9EF", legend="y")
p.line(x, f(x, 2), color="#F92672", line_width=3, legend="Actual", line_dash="dashed")

p.line(x, predict(x, x, y, h_optimal), color="#A6E22E", line_width=1, legend="y_hat (h={})".format(h_optimal))
    
p.title.text = "Kernel Regression (Gaussian)"
p.xaxis.axis_label = "x"
p.yaxis.axis_label = "f(x)"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

## k-Fold Cross Validation 

In [14]:
def split_k_fold(x, y, folds):
    if len(x) != len(y):
        raise ValueError("X and Y Should have same length")
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    split_size = len(x) / folds
    return np.array([x[n * split_size:(n + 1) * split_size] for n in np.arange(folds)]), np.array(
        [y[n * split_size:(n + 1) * split_size] for n in np.arange(folds)])

In [15]:
num_folds = 4

In [16]:
num_tries = 10

In [17]:
fold_indices  = np.arange(num_folds)
mse_values = []

for h in h_range:
    print("h = {}".format(h))
    trial_mses = []
    for trial in np.arange(num_tries):
        x_splits, y_splits = split_k_fold(x, y, num_folds)
        mses = []
        for idx in fold_indices:
            test_idx = idx
            train_idx = np.setdiff1d(fold_indices, [idx])
            train_x, test_x, train_y, test_y = (np.concatenate(x_splits[train_idx]), 
                                                x_splits[test_idx], 
                                                np.concatenate(y_splits[train_idx]), 
                                                y_splits[test_idx])
            test_y_hat = predict(test_x, train_x, train_y, h)
            mses.append(np.mean(np.power(test_y_hat - test_y, 2)))
        trial_mses.append(np.mean(mses))
    mse_values.append(np.mean(trial_mses))

h = 0.01
h = 0.02
h = 0.03
h = 0.04
h = 0.05
h = 0.06
h = 0.07
h = 0.08
h = 0.09
h = 0.1
h = 0.11
h = 0.12
h = 0.13
h = 0.14
h = 0.15
h = 0.16
h = 0.17
h = 0.18
h = 0.19
h = 0.2


In [18]:
p = figure(plot_width=600, plot_height=300)
p.circle(x=h_range, y=mse_values, size=10, color="#66D9EF")
p.line(x=h_range, y=mse_values, color="#66D9EF", line_width=3)

p.title.text = "MSE vs Backwidth"
p.xaxis.axis_label = "Backwidth"
p.yaxis.axis_label = "MSE"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)

In [19]:
h_optimal = h_range[np.argmin(mse_values)]
print(h_optimal)

0.04


In [20]:
p = figure(plot_width=600, plot_height=600)
p.circle(x, y, size=10, alpha=0.2, color="#66D9EF", legend="y")
p.line(x, f(x, 2), color="#F92672", line_width=3, legend="Actual", line_dash="dashed")

p.line(x, predict(x, x, y, h_optimal), color="#A6E22E", line_width=1, legend="y_hat (h={})".format(h_optimal))
    
p.title.text = "Kernel Regression (Gaussian)"
p.xaxis.axis_label = "x"
p.yaxis.axis_label = "f(x)"

doc = curdoc()
doc.theme = plot_theme
doc.add_root(p)
show(p)